# Calculating Forest Loss (using Hansen)

In [18]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

## Loading & Clipping Imagery

In [3]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Import the Forest loss image
flossHansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_hansen = ee.ImageCollection("UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1").mosaic().selfMask()
primaryForest_hansen_COL = primaryForest_hansen.clip(colombiaMpios)

# Clip the Forest loss image to Colombia's boundaries
flossHansen_COL = flossHansen.clip(colombiaMpios)

# Visualization parameters for the map
# viz_params = {
#     'min':1,  # Minimum value for visualization
#     'max': 1,  # Maximum value for visualization
#     'palette': ['008000']  # Color gradient
# }

# Initialize the map using geemap
# Map = geemap.Map(center=[4.0, -72.0], zoom=5)  # Center roughly in Colombia with an appropriate zoom level
# Map.addLayer(primaryForest_hansen_COL, viz_params, "Primary Tropical Forests")
# Map


In [22]:
# Create a mask where fcover is equal to 1.
primaryMask = primaryForest_hansen_COL.eq(1)

# Masking the floss imagery using the primary mask (cover equals 1)
flossHansen_COLmasked = flossHansen_COL.updateMask(primaryMask)

# Select the 'lossyear' band from the masked forest loss image
loss_year = flossHansen_COLmasked.select(['lossyear'])


# # Visualization parameters
# primary_forest_viz = {
#     'min': 1,
#     'max': 1,
#     'palette': ['green']  # Primary forest is displayed in green
# }

# loss_year_viz = {
#     'min': 1,
#      'max': 1,
#   'palette': ['red']
# }

# # Initialize the map using geemap
# Map = geemap.Map(center=[4.0, -72.0], zoom=6)  # Center roughly in Colombia

# # Add the Primary Tropical Forests layer
# Map.addLayer(primaryForest_hansen_COL, primary_forest_viz, "Primary Tropical Forests")

# # Add the Forest Loss for the specified year layer
# Map.addLayer(modified_loss_year, loss_year_viz, f"Forest Loss {year}")

# # Display the map
# Map

Map(center=[4.0, -72.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

<IPython.core.display.Javascript object>

vis_params = {'bands': ['lossyear'], 'palette': ['#ffff00', '#ff0000'], 'min': 0.9999999999999999, 'max': 120.0}


<IPython.core.display.Javascript object>

vis_params = {'bands': ['lossyear'], 'palette': ['#ffff00', '#ff0000'], 'min': 0.9999999999999999, 'max': 120.0}


In [41]:
# Create a mask for the current year
year = 11  # Replace with the desired year
loss_year_mask = loss_year.eq(year)

# Calculate the loss area for the current year in square meters
loss_area_image = loss_year_mask.multiply(ee.Image.pixelArea())

# Calculate the total loss area per municipality
def calculate_loss_area(feature):
    # Use reduceRegion to calculate the sum of the loss area within the feature's geometry
    loss_area = loss_area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('lossyear')  # Replace 'lossyear' with the correct property name if different
    
     # Dynamically name the loss area property
    loss_area_property_name = f'lossarea{year}'

    # Return a new feature with only the 'lossArea' and 'codmpio' properties
    return ee.Feature(None, {
        loss_area_property_name: loss_area,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
        'year': year  # Add the year as a property
    })

year_results = colombiaMpios.map(calculate_loss_area)

print(year_results.first().getInfo())




{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea11': 0, 'year': 11}}


In [42]:
# Convert the FeatureCollection to a list of features
features = year_results.getInfo()['features']

# Extract properties from each feature
data = [feature['properties'] for feature in features]

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
print(df.head())



   codmpio     lossarea11  year
0     5002       0.000000    11
1     5004   40580.973274    11
2    50006   62420.537476    11
3    27006  390131.367201    11
4    41006  506806.025966    11


In [44]:
# Calculate basic statistics
summary_stats = {
    'N': len(df),  # Number of observations
    'mean': df['lossarea11'].mean(),  # Mean
    'sd': df['lossarea11'].std(),  # Standard deviation
    'min': df['lossarea11'].min(),  # Minimum
    'p50': df['lossarea11'].median(),  # Median (50th percentile)
    'max': df['lossarea11'].max()  # Maximum
}

# Convert to a DataFrame for better visualization
summ_df = pd.DataFrame(summary_stats, index=[0])

# Display the summary statistics
print(summ_df)

      N          mean            sd  min         p50           max
0  1120  647323.42655  5.368329e+06  0.0  1773.97818  1.063543e+08
